In [ ]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands


In [ ]:

# For static images:
IMAGE_FILES = ["download.png"]
with mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=2,
    min_detection_confidence=0.5) as hands:
  for idx, file in enumerate(IMAGE_FILES):
    # Read an image, flip it around y-axis for correct handedness output (see
    # above).
    image = cv2.flip(cv2.imread(file), 1)
    # Convert the BGR image to RGB before processing.
    results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    # Print handedness and draw hand landmarks on the image.
    print('Handedness:', results.multi_handedness)
    if not results.multi_hand_landmarks:
      continue
    image_height, image_width, _ = image.shape
    annotated_image = image.copy()
    for hand_landmarks in results.multi_hand_landmarks:
      # print('hand_landmarks:', hand_landmarks)
      # print('hand:',results.multi_handedness[hand_index].classification[0].label)
      # print(
      #     f'Index finger tip coordinates: (',
      #     f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * image_width}, '
      #     f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * image_height})'      )
      mp_drawing.draw_landmarks(
          annotated_image,
          hand_landmarks,
          mp_hands.HAND_CONNECTIONS,
          mp_drawing_styles.get_default_hand_landmarks_style(),
          mp_drawing_styles.get_default_hand_connections_style())
    cv2.imwrite(
        '/tmp/annotated_image' + str(idx) + '.png', cv2.flip(annotated_image, 1))
    # Draw hand world landmarks.
    if not results.multi_hand_world_landmarks:
      continue
    for hand_world_landmarks in results.multi_hand_world_landmarks:
      mp_drawing.plot_landmarks(
        hand_world_landmarks, mp_hands.HAND_CONNECTIONS, azimuth=5)


In [18]:
VIDEO_FILES = [r"C:\Users\ranjan.patra\OneDrive - Lingaro Sp. z o. o\DATA\IITJ\Course\Projects\ISL\ISL_CSLRT_Corpus\Videos_Sentence_Level\are you free today\free (4).mp4"]

In [ ]:
import cv2
import mediapipe as mp
import pandas as pd

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# Create an empty DataFrame to store the data
df = pd.DataFrame(columns=['frame_number', 'hand_label', 'hand_index', 'landmark_index', 'landmark_name', 'x', 'y', 'z', 'global_x', 'global_y', 'global_z'])

# For video files:
#VIDEO_FILES = ["input.mp4"]

with mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.5) as hands:
  
  for video_file in VIDEO_FILES:
    cap = cv2.VideoCapture(video_file)
    frame_number = 0

    while cap.isOpened():
      success, image = cap.read()
      if not success:
        break

      # Flip the image horizontally for a later selfie-view display, and convert
      # the BGR image to RGB.
      image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
      # To improve performance, optionally mark the image as not writeable to
      # pass by reference.
      image.flags.writeable = False
      results = hands.process(image)

      # Draw the hand annotations on the image.
      image.flags.writeable = True
      image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
      if results.multi_hand_landmarks:
        for hand_index, hand_landmarks in enumerate(results.multi_hand_landmarks):
          # Determine if left or right hand
          if results.multi_handedness[hand_index].classification[0].label == 'Right':
            hand_label = 'Right'
          else:
            hand_label = 'Left'
          
          mp_drawing.draw_landmarks(
              image,
              hand_landmarks,
              mp_hands.HAND_CONNECTIONS,
              mp_drawing_styles.get_default_hand_landmarks_style(),
              mp_drawing_styles.get_default_hand_connections_style())
          
          # Get the wrist landmark
          wrist = hand_landmarks.landmark[0]
          
          # Append the hand landmarks to the DataFrame
          for landmark_index, landmark in enumerate(hand_landmarks.landmark):
            df = df.append({
                'frame_number': frame_number,
                'hand_label': hand_label,
                'hand_index': hand_index,
                'landmark_index': landmark_index,
                'landmark_name': mp_hands.HandLandmark(landmark_index).name,
                'x': landmark.x,
                'y': landmark.y,
                'z': landmark.z,
                'global_x': landmark.x - wrist.x,
                'global_y': landmark.y - wrist.y,
                'global_z': landmark.z - wrist.z
            }, ignore_index=True)

      cv2.imshow('MediaPipe Hands', image)
      if cv2.waitKey(5) & 0xFF == 27:
        break

      frame_number += 1

    cap.release()

cv2.destroyAllWindows()

# Print the DataFrame
print(df)

In [ ]:
display(df.head(50))

In [ ]:
import os
import fnmatch

def list_video_files(root_folder, extensions=["*.mp4", "*.mkv", "*.avi"]):
    video_files = []
    for folder_path, _, files in os.walk(root_folder):
        for extension in extensions:
            for file_name in fnmatch.filter(files, extension):
                video_files.append(os.path.join(folder_path, file_name))
    return video_files

# Example usage
folder_path = r"C:\Users\ranjan.patra\OneDrive - Lingaro Sp. z o. o\DATA\IITJ\Course\Projects\ISL\ISL_CSLRT_Corpus\Videos_Sentence_Level"
video_files_list = list_video_files(folder_path)

for file_path in video_files_list:
    print(file_path)
